In [1]:
import sys
import requests
from PyQt5.QtWidgets import (QApplication, QMainWindow, 
                            QVBoxLayout, QWidget,
                            QTextEdit, QLineEdit, QPushButton, 
                            QHBoxLayout, QLabel)
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QFont, QSyntaxHighlighter, QTextCharFormat, QColor, QTextDocument
import re
import certifi

class PythonHighlighter(QSyntaxHighlighter):
    def __init__(self, document):
        super().__init__(document)
        self.highlight_rules = []
        
        # Keywords
        keywords = [
            'and', 'as', 'assert', 'break', 'class', 'continue', 'def', 'del',
            'elif', 'else', 'except', 'False', 'finally', 'for', 'from', 'global',
            'if', 'import', 'in', 'is', 'lambda', 'None', 'nonlocal', 'not', 'or',
            'pass', 'raise', 'return', 'True', 'try', 'while', 'with', 'yield'
        ]
        
        keyword_format = QTextCharFormat()
        keyword_format.setForeground(QColor(88, 129, 87))
        keyword_format.setFontWeight(QFont.Bold)
        self.highlight_rules.extend((fr'\b{word}\b', keyword_format) for word in keywords)
        
        # Strings
        string_format = QTextCharFormat()
        string_format.setForeground(QColor(206, 145, 120))
        self.highlight_rules.append((r'"[^"\\]*(\\.[^"\\]*)*"', string_format))
        self.highlight_rules.append((r"'[^'\\]*(\\.[^'\\]*)*'", string_format))
        
        # Comments
        comment_format = QTextCharFormat()
        comment_format.setForeground(QColor(99, 99, 99))
        comment_format.setFontItalic(True)
        self.highlight_rules.append((r'#[^\n]*', comment_format))
        
        # Numbers
        number_format = QTextCharFormat()
        number_format.setForeground(QColor(144, 122, 214))
        self.highlight_rules.append((r'\b[0-9]+\b', number_format))
        
        # Functions
        function_format = QTextCharFormat()
        function_format.setForeground(QColor(32, 121, 255))
        self.highlight_rules.append((r'\bdef\s+(\w+)', function_format))
        
    def highlightBlock(self, text):
        for pattern, format in self.highlight_rules:
            for match in re.finditer(pattern, text):
                start, end = match.span()
                self.setFormat(start, end - start, format)

class CodeChatbotGUI(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setup_ui()
        self.api_url = "https://8775d0cf1209.ngrok-free.app/predict"  # Replace
        
    def setup_ui(self):
        self.setWindowTitle("Python Code Generator")
        self.resize(800, 600)
        
        # Central Widget
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        layout = QVBoxLayout()
        
        # Title
        title = QLabel("Python Code Generator")
        title.setFont(QFont("Arial", 16, QFont.Bold))
        title.setAlignment(Qt.AlignCenter)
        layout.addWidget(title)
        
        # Chat Display (for code)
        self.code_display = QTextEdit()
        self.code_display.setReadOnly(True)
        self.code_display.setFont(QFont("Consolas", 12))
        self.highlighter = PythonHighlighter(self.code_document())
        layout.addWidget(self.code_display)
        
        # Input Area
        input_layout = QHBoxLayout()
        
        self.user_input = QLineEdit()
        self.user_input.setPlaceholderText("Describe what Python code you need...")
        self.user_input.setFont(QFont("Arial", 12))
        self.user_input.returnPressed.connect(self.send_request)
        input_layout.addWidget(self.user_input)
        
        self.send_button = QPushButton("Generate")
        self.send_button.setFont(QFont("Arial", 12))
        self.send_button.clicked.connect(self.send_request)
        input_layout.addWidget(self.send_button)
        
        layout.addLayout(input_layout)
        
        central_widget.setLayout(layout)
    
    def code_document(self):
        return self.code_display.document()
    
    def send_request(self):
        request_text = self.user_input.text().strip()
        if not request_text:
            return
            
        self.code_display.append(f"<b>You:</b> {request_text}")
        self.user_input.clear()
        self.user_input.setEnabled(False)
        self.send_button.setEnabled(False)
        
        try:
            response = requests.post(
                self.api_url,
                json={"text": request_text},
                verify=False,
                timeout=30
            )
            
            if response.status_code == 200:
                python_code = response.json().get('response', '')
                self.display_code(python_code)
            else:
                self.code_display.append(f"<font color='red'>Error: {response.text}</font>")
                
        except Exception as e:
            self.code_display.append(f"<font color='red'>Error: {str(e)}</font>")
        finally:
            self.user_input.setEnabled(True)
            self.send_button.setEnabled(True)
            self.user_input.setFocus()
    
    def display_code(self, code):
        self.code_display.append("<b>Generated Code:</b>")
        self.code_display.append("```python")
        self.code_display.append(code)
        self.code_display.append("```\n")
        self.code_display.verticalScrollBar().setValue(
            self.code_display.verticalScrollBar().maximum()
        )

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = CodeChatbotGUI()
    window.show()
    sys.exit(app.exec_())

C:\Users\ajeait\AppData\Roaming\Python\Python312\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '8775d0cf1209.ngrok-free.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


SystemExit: 0

C:\Users\ajeait\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
